Author: Soumyadip Mukherjee
Team Name: Soumo275
Team Members: Soumyadip Mukherjee
Leaderboard Rank: 38

In [ ]:
import zipfile
import os

zip_path = "/content/soil-classification-part-2.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("soil-classification")

In [ ]:
import os
from pathlib import Path
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import joblib

# Configuration
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
scaler = StandardScaler()
img_size = (224, 224)
train_path = '/content/soil-classification/soil_competition-2025/train'

# Feature extraction
def get_features(img_path):
    img = load_img(img_path, target_size=img_size)
    x = img_to_array(img).reshape(1, *img_size, 3)
    return model.predict(x).flatten()

# Process images
features = []
for img_file in Path(train_path).glob('*'):
    if img_file.suffix.lower() in ('.jpg', '.jpeg', '.png'):
        try:
            features.append(get_features(str(img_file)))
        except Exception as e:
            print(f"Error: {img_file.name} - {str(e)}")

# Train model
X = scaler.fit_transform(features)
ocsvm = OneClassSVM(kernel='rbf', nu=0.1).fit(X)
joblib.dump((ocsvm, scaler), 'soil_model.pkl')
print(f"Trained on {len(X)} samples | Model saved")
